In [95]:
# Goal: Predict the win rate of a team/combination of agents given minimal data
# such as statistics for the first few rounds (or just the team composition)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

engine = create_engine("sqlite:///data/valorant.sqlite")

scoreboard = pd.read_sql_table("Game_Scoreboard", con=engine)
rounds = pd.read_sql_table("Game_Rounds", con=engine)
games = pd.read_sql_table("Games", con=engine)

engine.dispose()

In [96]:
print("Scoreboard:")
print(scoreboard.head())
print("Rounds:")
print(rounds.head())
print("Games:")
print(games.head())

Scoreboard:
  GameID PlayerID PlayerName TeamAbbreviation    Agent    ACS  Kills  Deaths  \
0  60894     8419     Reduxx             Boos     jett  313.0   24.0    10.0   
1  60894      466     ChurmZ             Boos  chamber  227.0   16.0    10.0   
2  60894     3712   diaamond             Boos     sova  226.0   17.0     9.0   
3  60894     5099     Boltzy             Boos    viper  218.0   17.0    12.0   
4  60894     3983     Virtyy             Boos     skye   80.0    5.0    13.0   

   Assists  PlusMinus  ...  Num_4Ks  Num_5Ks  OnevOne  OnevTwo  OnevThree  \
0      3.0       14.0  ...      2.0      0.0      1.0      0.0        0.0   
1      7.0        6.0  ...      0.0      0.0      0.0      0.0        0.0   
2      8.0        8.0  ...      0.0      0.0      1.0      0.0        0.0   
3      2.0        5.0  ...      0.0      0.0      1.0      0.0        0.0   
4      3.0       -8.0  ...      0.0      0.0      0.0      0.0        0.0   

   OnevFour  OnevFive  Econ  Plants  Defuses

__Small-scale test for win-rate based on team composition:__

In [97]:
# Desired DataFrame structure:
# Index: GameID, Columns: Team1, Team1 agents, Team2, Team2 agents, Winner

print(len(scoreboard))
#scoreboard_grouped = scoreboard.copy().groupby('GameID')
game_ids_s = scoreboard['GameID'].unique()
print(len(game_ids_s))
#print(len(scoreboard_grouped))
print("------")
print(len(rounds))
print("------")

games = games[games['GameID'].isin(game_ids_s)]

game_ids_g = games['GameID'].unique()
scoreboard = scoreboard[scoreboard['GameID'].isin(game_ids_g)]

print(len(scoreboard))
print(len(games))
#print(scoreboard.head())

157939
15878
------
15531
------
157939
15878


In [94]:
# Step 1: Group agents by GameID and TeamAbbreviation into sets
team_compositions = pd.pivot_table(
    scoreboard,
    values='Agent',  # The column containing the agents
    index=['GameID', 'TeamAbbreviation'],  # Rows will be grouped by GameID and TeamID
    aggfunc=lambda x: set(x)  # Combine agent names into a set for each team
)

# Filter GameIDs with exactly two teams
team_counts = team_compositions.groupby(level='GameID').size()
valid_game_ids = team_counts[team_counts == 2].index
team_compositions = team_compositions.loc[valid_game_ids]

# Step 2: For each GameID, convert team abbreviation to full team name
teams = games[['GameID', 'Team1', 'Team2', 'Winner']]
team_map = games.set_index("GameID")[["Team1", "Team2"]]

merged = team_compositions.merge(games, how="left", on="GameID")

# Step 2: Assign positions in each group by GameID
merged["RowPosition"] = merged.groupby("GameID").cumcount()

# Step 3: Update TeamAbbreviation based on RowPosition
merged.loc[merged["RowPosition"] == 0, "TeamAbbreviation"] = merged["Team1"]
merged.loc[merged["RowPosition"] == 1, "TeamAbbreviation"] = merged["Team2"]

# Add column for winner
merged["Winner1"] = merged["TeamAbbreviation"] == merged["Winner"]

# Drop unnecessary columns like Team1, Team2, and RowPosition
team_compositions = merged[["GameID", "TeamAbbreviation", "Agent", "Winner1"]]


print(team_compositions.head(10))
print(teams[teams['GameID'] == '10004'])
print(len(team_compositions.index.get_level_values(0).unique()) / 2)
print(len(teams['GameID']))

  GameID   TeamAbbreviation                                  Agent  Winner1
0  10003       Bren Esports    {phoenix, omen, sova, cypher, raze}    False
1  10003  Attack All Around      {reyna, omen, sova, cypher, raze}     True
2  10004       Bren Esports      {reyna, omen, sova, cypher, jett}     True
3  10004  Attack All Around     {killjoy, reyna, omen, sova, jett}    False
4  10005       Bren Esports     {omen, cypher, jett, raze, breach}    False
5  10005  Attack All Around    {killjoy, omen, jett, raze, breach}     True
6  10006             HSDIRR     {phoenix, omen, viper, raze, sage}     True
7  10006           MJAOMODE  {killjoy, phoenix, omen, viper, raze}    False
8  10007             HSDIRR     {phoenix, omen, viper, jett, sage}    False
9  10007           MJAOMODE  {killjoy, phoenix, omen, viper, raze}     True
      GameID         Team1              Team2        Winner
12733  10004  Bren Esports  Attack All Around  Bren Esports
14771.0
15878


In [17]:
# Part 1 goal:
# Find the chance of a chosen combination of agents winning
# against another combination of agents. First step will be
# finding overall win rate for a combination of agents.

wins = games['Winner']
wins.index = games['GameID']
print(wins.head())

team_compositions = pd.pivot_table(
    scoreboard,
    values='Agent',  # The column containing the agents
    index=['GameID', 'TeamAbbreviation'],  # Rows will be grouped by GameID and TeamID
    aggfunc=lambda x: set(x)  # Combine agent names into a set for each team
)
team_compositions.index = team_compositions.index.set_levels(
    team_compositions.index.levels[0].astype(int), level='GameID'
)

print(team_compositions.head())
print(team_compositions.loc[60903])
#print(wins.iloc[10003], wins.iloc[10004])

# Team1 is top TeamAbb, Team2 is bottom

GameID
60894    Booster Seat Gaming
60895               Pho Real
60896    Booster Seat Gaming
60924         Bjor's Kittens
60925         Bjor's Kittens
Name: Winner, dtype: object
                                                       Agent
GameID TeamAbbreviation                                     
10003  AAA               {sova, raze, phoenix, cypher, omen}
       BRN                 {sova, raze, cypher, reyna, omen}
10004  AAA                 {sova, cypher, reyna, jett, omen}
       BRN                {sova, reyna, jett, omen, killjoy}
10005  AAA                {breach, raze, cypher, jett, omen}
                                                 Agent
TeamAbbreviation                                      
RAD               {viper, astra, sova, reyna, chamber}
same                {viper, sova, cypher, reyna, jett}


In [20]:
team_comps_unique = []
t = 0
for i in team_compositions.index:
    if team_compositions.loc[i]['Agent'] not in team_comps_unique:
        team_comps_unique.append(team_compositions.loc[i]['Agent'])

In [72]:
print(len(team_comps_unique))
print(team_comps_unique[-1])

# Initialize 'who_won' DataFrame based on the team_compositions index
who_won = pd.DataFrame(index=['GameID', 'Team'], columns=["Winner"], dtype=bool)
who_won['Winner'] = True

# Group the DataFrame by GameID
grouped = who_won.groupby(level="GameID")

# Select the 0th row (first row in each group)
first_row_idx = grouped.nth(0).index

# Select the 1st row (second row in each group)
second_row_idx = grouped.nth(1).index

#who_won.loc[first_row_idx, "Winner"] = True if games[]

# for _, row in games.iterrows():
#     game_id = int(row["GameID"])
#     winner = row["Winner"]
#
#     if len(who_won.loc[game_id]) < 2:
#         who_won = who_won.drop(game_id)
#         continue
#     first_row = who_won.loc[game_id].iloc[0].name  # Abbreviation for Team1
#
#     second_row = who_won.loc[game_id].iloc[1].name  # Abbreviation for Team2
#
#     # Update the Winner column directly using index
#     if row["Team1"] == winner:  # Team1 won
#         who_won.at[(game_id, first_row), "Winner"] = True
#         who_won.at[(game_id, second_row), "Winner"] = False
#     elif row["Team2"] == winner:  # Team2 won
#         who_won.at[(game_id, first_row), "Winner"] = False
#         who_won.at[(game_id, second_row), "Winner"] = True


#print(who_won.loc[57193])
games[games['GameID'] == "57193"]

1456
{'sage', 'raze', 'cypher', 'reyna', 'jett'}


,GameID,MatchID,Map,Team1ID,Team2ID,Team1,Team2,Winner,Team1_TotalRounds,Team2_TotalRounds,...,Team1_FullBuyWon,Team2_PistolWon,Team2_Eco,Team2_EcoWon,Team2_SemiEco,Team2_SemiEcoWon,Team2_SemiBuy,Team2_SemiBuyWon,Team2_FullBuy,Team2_FullBuyWon
1375,57193,50407,TBD,6748,6737,ANYWAY,Claquettes cahussettes,ANYWAY,13,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
agents = scoreboard['Agent'].unique()
print(agents, len(agents))

import pandas as pd

# Example: Extract unique agents (already done earlier)
agents = scoreboard['Agent'].unique()


# Use team_compositions (teams with sets of agents) to create one-hot encoding
def one_hot_encode_team_compositions(team_compositions, all_agents):
    # 'all_agents' defines the feature space (columns)
    encoded_data = []
    team_index = []

    for idx, agents_set in team_compositions.items():
        # For each team composition, generate a binary array for all agents
        encoded_row = [1 if agent in agents_set else 0 for agent in all_agents]
        encoded_data.append(encoded_row)
        team_index.append(idx)  # Keep track of MultiIndex values (GameID, TeamAbbreviation)

    # Create a DataFrame with one-hot encoded rows, indexed by (GameID, TeamAbbreviation)
    one_hot_df = pd.DataFrame(
        encoded_data,
        columns=all_agents,
        index=pd.MultiIndex.from_tuples(team_index, names=team_compositions.index.names)
    )
    return one_hot_df


# Apply the one-hot encoding function
one_hot_team_comps = one_hot_encode_team_compositions(team_compositions['Agent'], agents)

print(one_hot_team_comps.head())
print("THIS IS IT ^^^")

# print(team_compositions.info())
# print(one_hot_team_comps.info())

['jett' 'chamber' 'sova' 'viper' 'skye' 'astra' 'raze' 'sage' 'kayo'
 'killjoy' 'reyna' 'cypher' 'breach' 'omen' 'brimstone' '' 'phoenix'
 'yoru'] 18
                         jett  chamber  sova  viper  skye  astra  raze  sage  \
GameID TeamAbbreviation                                                        
10003  AAA                  0        0     1      0     0      0     1     0   
       BRN                  0        0     1      0     0      0     1     0   
10004  AAA                  1        0     1      0     0      0     0     0   
       BRN                  1        0     1      0     0      0     0     0   
10005  AAA                  1        0     0      0     0      0     1     0   

                         kayo  killjoy  reyna  cypher  breach  omen  \
GameID TeamAbbreviation                                               
10003  AAA                  0        0      0       1       0     1   
       BRN                  0        0      1       1       0     1   
10004

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

reg_model = GradientBoostingRegressor()
reg_model.fit(X_train, y_train)
